<a href="https://colab.research.google.com/github/weihanchen/google-colab-python-learn/blob/main/jupyter-examples/whisper/whisper_acc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 【語音辨識 - Whisper】 準確與否需要有一把 📏尺來衡量


前面我們介紹了幾個關於Whisper的基本概念，這裡附上 [🚀傳送門](https://vocus.cc/article/644526c8fd89780001ffdd9f) ，歡迎好好閱讀一番，但我們除了學會如何用語音辨識的工具之外，「準確率」對我們來說也是一個非常重要的一環，但我們究竟應該要如何評估所謂的準確率呢？ 不知道沒關係，當您看完這個篇章就能夠學會如何計算文字的「字元錯誤率」、「字詞錯誤率」...，非常值得您細細品嘗與學習，就讓我們往下一步步的完成評估準確率的程序吧！

這次的評估工具我們會使用jiwer這一套來進行說明，它支援了多種的計算方式，包括： WER、CER、MER...等，那這些計算方式各有什麼不同呢？ 就讓我們繼續看下去吧！

## 安裝套件

In [ ]:
# 錯誤率計算工具
!pip install jiwer

# 移除掉與語音辨識套件相同名稱的套件
# !pip uninstall whisper

# 語音辨識ASR
!pip install -U openai-whisper

# Hugging Face資料集函式庫
!pip install datasets

# 斷詞器
!pip install jiaba

## 有哪些不同的計算方式呢？

### 以「詞」為單位進行計算

我們先來看看詞的計算結果如下：

In [ ]:


reference = "今天 天氣 很好 嗎"
hypothesis = "今天 天氣 很 好 啊"

out = jiwer.process_words(reference, hypothesis)
print(jiwer.visualize_alignment(out))

sentence 1
REF: 今天 天氣 * 很好 嗎
HYP: 今天 天氣 很  好 啊
           I  S S

number of sentences: 1
substitutions=2 deletions=0 insertions=1 hits=2

mer=60.00%
wil=80.00%
wip=20.00%
wer=75.00%



#### 詞錯誤率 Word Error Rate(WER)
WER是以「詞」為單位進行計算，它用來衡量句子中有多少詞彙需要進行修改才能和正確答案一樣。

```bash
公式: (S + D + I) / (H + S + D)
計算過程: (2 + 0 + 1) / (2 + 2 + 0)
3 / 4 ≈ 75%。
```

💡 既然是以`詞`為單位的話，那麼我們的答案與辨識結果請先進行斷詞(通常用空白隔開)， 標點符號也是考量的因素之一喔。

#### 平均錯誤率 Mean Error Rate(MER)
這項指標與WER主要差別在於分母的部分尚未將`Insertion`給考量進來計算，因為它衡量的不僅是詞彙層級，而是句子層級，因此會更加全面。

```bash
公式： (S + D + I) / (H + S + D + I)
計算過程： (2 + 0 + 1) / (2 + 2 + 0 + 1)

3 / 5 ≈ 60%
```

#### 詞保留率 Word Information Preservation(WIP)
這項指標主要在評估我們的辨識結果究竟有多少比例的字詞是一模一樣完全正確的。

```bash
num_rf_words = 正確答案字詞數 = 4
num_hp_words = 辨識結果字詞數 = 5
公式： (H / num_rf_words) * (H / num_hp_words)
計算過程: (2 / 4) * (2 / 5)
0.5 * 0.4 ≈ 20%
```
#### 詞漏失率 Word Information Lost(WIL)
既然有詞的保留率，那麼相反的就是漏失率，因此上述的結果得出之後，用1減去保留率就是漏失率，可以粗略的評估總共漏失了多少比率。
```bash
公式: 1 - wip
1 - 0.2 ≈ 80%
```

## 以「字元」為單位進行計算

### 字元錯誤率 Character Error Rate(CER)
CER是以「字元」為單位進行計算，底下的例子以「字元」為單位會發現有1個substitution，因此總共7個字元錯了1個等於：

```
1 / 7 = 14.28%
```

💡 既然是以`字元`為單位的話，那麼我們的答案與辨識結果請將空白給去除， 才不會也被計算進去喔， 甚至標點符號...等都是考量的因素之一。

In [ ]:
import jiwer

reference = "今天天氣很好嗎"
hypothesis = "今天天氣很好啊"

output = jiwer.process_characters(reference, hypothesis)
print(jiwer.visualize_alignment(output))

sentence 1
REF: 今天天氣很好嗎
HYP: 今天天氣很好啊
           S

number of sentences: 1
substitutions=1 deletions=0 insertions=0 hits=6

cer=14.29%



## 動動手使用Whisper語音辨識來計算一下正確率吧

首先我們先從Hugging Face找尋Common Voice的音檔：

https://huggingface.co/datasets/common_voice/viewer/zh-TW/train


### 接著我們使用Hugging Face的Datasets函式庫來進行操作

關於Datasets是什麼？ 歡迎參考：「[【Hugging Face】Ep.3 前往Datasets掏金趣](https://vocus.cc/article/64a2c62afd897800018a8185)」。

這邊會取測試集的第一筆做為我們的參考答案製作來源。

In [ ]:
from datasets import load_dataset

# 載入中文的資料集
ds = load_dataset("common_voice", name='zh-TW', split='test')

# 取第一筆做為本次的參考答案資料集
ref_data = ds[0]

ref_data

/root/.cache/huggingface/modules/datasets_modules/datasets/common_voice/220833898d6a60c50f621126e51fb22eb2dfe5244392c70dccd8e6e2f055f4bf/common_voice.py:634: FutureWarning: 
            This version of the Common Voice dataset is deprecated.
            You can download the latest one with
            >>> load_dataset("mozilla-foundation/common_voice_11_0", "en")
            
  warnings.warn(


{'client_id': '7fbf85e97244d202891c843cad48ee736b12cf6ceddc165ddd821804af13e01852da09faf0f3b647a11c1d0fde52b4efa9f71e9852d5acf16776d022d983bae4',
 'path': '/root/.cache/huggingface/datasets/downloads/extracted/0cd3800424a383996b39a64547fd7ea9852d200cc41113eebeb6f790cf74e9ca/cv-corpus-6.1-2020-12-11/zh-TW/clips/common_voice_zh-TW_17369547.mp3',
 'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/0cd3800424a383996b39a64547fd7ea9852d200cc41113eebeb6f790cf74e9ca/cv-corpus-6.1-2020-12-11/zh-TW/clips/common_voice_zh-TW_17369547.mp3',
  'array': array([ 0.00000000e+00,  1.77818101e-13,  2.42973935e-13, ...,
         -6.08271512e-06,  1.44303194e-05, -3.25221044e-05]),
  'sampling_rate': 48000},
 'sentence': '兩國總統都沒有直接通過電話',
 'up_votes': 3,
 'down_votes': 0,
 'age': 'thirties',
 'gender': 'male',
 'accent': '',
 'locale': 'zh-TW',
 'segment': "''"}

透過上述的操作我們可以得到幾個資訊：
- 語句: 並做出行動
- 音檔路徑: `/root/.cache/huggingface/datasets/downloads/extracted/0cd3800424a383996b39a64547fd7ea9852d200cc41113eebeb6f790cf74e9ca/cv-corpus-6.1-2020-12-11/zh-TW/clips/common_voice_zh-TW_17370757.mp3`

### 製作參考答案文字

首先我們對語句的部分進行斷詞，來製作參考答案的文字：

```python
並 做出 行動
```

In [ ]:
import jieba
_reference = ref_data['sentence']

reference = ' '.join(jieba.cut(_reference, cut_all=False, HMM=True))

reference # 兩國 總統 都 沒 有 直接 通過 電話


'兩國 總統 都 沒 有 直接 通過 電話'

### 接著來進行語音辨識

這邊我們使用`base`模型就好

In [ ]:
import whisper

model = whisper.load_model("base")

audio = ref_data['path']

result = model.transcribe(audio)

result

/usr/local/lib/python3.10/dist-packages/whisper/transcribe.py:114: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


{'text': '兩國總統都沒有直接通過電話。',
 'segments': [{'id': 0,
   'seek': 0,
   'start': 0.0,
   'end': 5.0,
   'text': '兩國總統都沒有直接通過電話。',
   'tokens': [50364,
    16313,
    8053,
    26575,
    33725,
    7182,
    6963,
    43297,
    19550,
    8816,
    20545,
    11103,
    1543,
    50614],
   'temperature': 0.0,
   'avg_logprob': -0.32310527165730796,
   'compression_ratio': 0.7924528301886793,
   'no_speech_prob': 0.031160296872258186}],
 'language': 'zh'}

### 最終我們透過jiwer計算一下錯誤率

正確率怎麼計算呢？ 1 - 錯誤率(WER、WIP、MER...)

P.S 看起來幾乎都沒有錯誤，猜測可能是common voice的語料已涵蓋在whisper的模型之中了...。

In [ ]:
import jiwer
hypothesis = result['text']

hypothesis = ' '.join(jieba.cut(hypothesis, cut_all=False, HMM=True))

out = jiwer.process_words(reference, hypothesis)
print(jiwer.visualize_alignment(out))

sentence 1
REF: 兩國 總統 都 沒 有 直接 通過 電話 *
HYP: 兩國 總統 都 沒 有 直接 通過 電話 。
                          I

number of sentences: 1
substitutions=0 deletions=0 insertions=1 hits=8

mer=11.11%
wil=11.11%
wip=88.89%
wer=12.50%

